## Multiclass Case

In [1]:
# libraries
import numpy as np
import cupy as cp

In [25]:
# other test variables, which are self parameters
# classes schould be given as int from 0 to k
n_total_ftr = 3
target_values = [0,1]
mu = cp.ones((n_total_ftr, len(target_values))) * 0
sigma = cp.ones((n_total_ftr, len(target_values))) * 1
penalty_s = 0.01
penalty_r = 0.01
epochs = 1
lr_mu = 1
lr_sigma = 1
n_mc_samples = 2 # monte carlo samples



In [26]:
def __softmax( x, y):

        """
        Update the distribution parameters mu and sigma by optimizing them in terms of the (log) likelihood.
        Here we assume a multinominal distributed target variable. We use a Multinominal model as our base model.
        Funciton with cupy functions.

        :param x: (np.ndarray) Batch of observations (numeric values only, consider normalizing data for better results)
        :param y: (np.ndarray) Batch of labels: type integer e.g. 0,1,2,3,4 etc.
        """
        global sigma
        global mu
        global epochs
        global lr_mu
        global lr_sigma
        global n_mc_samples
        if len(x.shape) != 2:
            x = x.reshape(1,len(x))
    
        observed_classes = np.unique(y)

        for obs_class in observed_classes:
            observations_index = np.where(y == obs_class)[0]
            x_obs = cp.array(x[observations_index])
            print("x:")
            print(x_obs)
            n_obs = len(x_obs)
            print("y: {}".format(obs_class))
            #print("obs_class: {}, n obs: {}".format(obs_class, n_obs))

            for epoch in range(epochs):
                    
                    # Iterative update of mu and sigma
                    try:
                        # o number of obs, l number of samples, j features,
                        # c classes
                        
                        # r shape: oxlxjxc
                        r = cp.random.randn(n_obs, n_mc_samples,
                                            n_total_ftr,
                                            len(target_values))

                        r[0,:,:,:] = cp.array(([[0.5,0.5],[0.5,0.5],[0.5,0.5]],[[1,1],[1,1],[1,1]]))
                        r.reshape(1,2,3,2)
                        print(r)
                        

                        # theta shape: oxlxjxc
                        theta = (r * sigma + mu)
                        print("theta:{}".format(theta.shape))
                        # eta shape: oxlxc
                        # multiply all ftr_cols with given ftr_vector x
                        print(theta)
                        eta = cp.einsum("oljc,oj->oljc", theta, x_obs) 

                        # sum up all theta^cl_j * x_tj so we got l samples
                        # for all c classes
                        eta = cp.einsum("oljc->olc", eta) 
                        print("eta: {}".format(eta.shape))
                        print(eta)

                        # get a for numerical stability, shape oxl
                        a = cp.amax(eta, axis=2) * -1
                        print("a: {}".format(a.shape))
                        print(a)

                        eta = cp.einsum("olc->col", eta) + a
                        eta = cp.einsum("col->olc", eta)
                        print("eta with a: {}".format(eta.shape))
                        print(eta)


                        eta = cp.exp(eta) # we only need them exp
                        print("finally eta: {}".format(eta.shape))
                        print(eta)

                        # eta_sum shape: oxl
                        eta_sum = cp.einsum("olc->ol", eta)
                        print("sums:{}".format(eta_sum.shape))
                        print(eta_sum)
                        
                        # calculate softmax only for all classes
                        # divide all etas by eta_sum
                        softmax_all = np.einsum("olc,ol->olc", eta, (1/eta_sum))

                        print("softmax: {}".format(softmax_all.shape))
                        
                        # marginal shape: o
                        marginal = cp.einsum("ol->o",
                                             softmax_all[:,:,obs_class]) / \
                                   n_mc_samples

                        print("marginal: {}".format(marginal.shape))
                        print(marginal)


                        # calculate softmax derivative to theta
                        softmax_c = softmax_all[:,:,obs_class]
                        print("sm_c:{}".format(softmax_c.shape))

                        # first calculate derivative for all as k != c
                        softmax_derivative = -1 * cp.einsum("oj,ol,olc->oljc",
                                                            (x_obs),
                                                            softmax_c,
                                                            softmax_all)
                        print("derivavtive: {}".format(softmax_derivative.shape))
                        print(softmax_derivative)

                        # then for observed class c
                        softmax_derivative_c = cp.einsum("oj,ol,ol->olj",
                                                         x_obs,
                                                         softmax_c,
                                                         (1-softmax_c))
                        print("derivative c: {}".format(softmax_derivative_c.shape))
                        print(softmax_derivative_c)

                        softmax_derivative[:,:,:,obs_class] = \
                            softmax_derivative_c
                        
                        print("end derivative: {}".format(softmax_derivative.shape))
                        print(softmax_derivative)

                        nabla_mu = cp.einsum("oljc->ojc", softmax_derivative) /\
                                   n_mc_samples

                        print("nabla_mu: {}".format(nabla_mu.shape))
                        print(nabla_mu)

                        
                        
                        
                        nabla_sigma = cp.einsum("oljc,oljc->ojc",
                                                softmax_derivative,r) / \
                                      n_mc_samples
                        print("nablu_sigma: {}".format(nabla_sigma.shape))
                        print(nabla_sigma)
                                    

                        nabla_mu = cp.einsum("ojc->jco", nabla_mu)
                        mu += lr_mu * \
                                                cp.einsum("jco->jc",
                                                          (nabla_mu/ marginal))
                        print("mu:")
                        print(mu)

                        nabla_sigma = cp.einsum("ojc->jco", nabla_sigma)
                        sigma += lr_sigma * \
                                                   cp.einsum("jco->jc",
                                                             (nabla_sigma / 
                                                             marginal))
                        print("sigma:")
                        print(sigma)

                    except TypeError as e:
                            raise TypeError('All features must be a numeric data type.') from e

In [27]:
#test runs
__softmax(np.array((1,2,3)),1)

x:
[[1 2 3]]
y: 1
[[[[0.5 0.5]
   [0.5 0.5]
   [0.5 0.5]]

  [[1.  1. ]
   [1.  1. ]
   [1.  1. ]]]]
theta:(1, 2, 3, 2)
[[[[0.5 0.5]
   [0.5 0.5]
   [0.5 0.5]]

  [[1.  1. ]
   [1.  1. ]
   [1.  1. ]]]]
eta: (1, 2, 2)
[[[3. 3.]
  [6. 6.]]]
a: (1, 2)
[[-3. -6.]]
eta with a: (1, 2, 2)
[[[0. 0.]
  [0. 0.]]]
finally eta: (1, 2, 2)
[[[1. 1.]
  [1. 1.]]]
sums:(1, 2)
[[2. 2.]]
softmax: (1, 2, 2)
marginal: (1,)
[0.5]
sm_c:(1, 2)
derivavtive: (1, 2, 3, 2)
[[[[-0.25 -0.25]
   [-0.5  -0.5 ]
   [-0.75 -0.75]]

  [[-0.25 -0.25]
   [-0.5  -0.5 ]
   [-0.75 -0.75]]]]
derivative c: (1, 2, 3)
[[[0.25 0.5  0.75]
  [0.25 0.5  0.75]]]
end derivative: (1, 2, 3, 2)
[[[[-0.25  0.25]
   [-0.5   0.5 ]
   [-0.75  0.75]]

  [[-0.25  0.25]
   [-0.5   0.5 ]
   [-0.75  0.75]]]]
nabla_mu: (1, 3, 2)
[[[-0.25  0.25]
  [-0.5   0.5 ]
  [-0.75  0.75]]]
nablu_sigma: (1, 3, 2)
[[[-0.1875  0.1875]
  [-0.375   0.375 ]
  [-0.5625  0.5625]]]
mu:
[[-0.5  0.5]
 [-1.   1. ]
 [-1.5  1.5]]
sigma:
[[ 0.625  1.375]
 [ 0.25   1.75 ]
 [